### Neshyba, 2023


# A kinetic-molecular approach to reversible expansion of an ideal gas

## Introduction
This CGI is all about exploring the question, what's going on at the molecular level when we carry out an isothermal reversible expansion of a gas? Why, for example, does heat go in at all, and why does is that amount of heat equal to $nRT \ ln \bigl ({V_{final} \over V_{initial}} \bigr )$? Our basic premise here is that if we think about this heat as compensatation for the loss of speed that molecules undergo when they hit a slowly receding wall, then we have achieved a degree of molecular-level insight. 


## A hybrid (numerical+theoretical) kinetic-molecular approach

We'll begin with the result of an analysis of the physics of objects slamming against a receding wall. If that process is elastic (as Physicists are wont to insist on), one can show that those objects will bounce back with their speed reduced by $2v_{wall}$. That reduction in speed, in turn, implies the kinetic energy of those objects (OK, molecules) will also be reduced. If we want our process to be isothermal, then we can say that this reduction in kinetic energy would have to be compensated by an equal inflow of heat from the surroundings. 

Kinetic-molecular theory says that this inflow of heat is given by

$$
dq_{kinetic} = -Z \times \delta KE \times A \times dt \ \ \ \ (1)
$$

This is a complex expression, but we hope you can see why it has to work, based on the factors that go into it:

- $Z$ is the frequency with which molecules hit our wall (typical units: mole / meter$^2$ / second). Kinetic theory says $Z = {1 \over 2} \rho \times v$, where $\rho$ is the number density of the gas (mol / m$^2$), the factor ${1 \over 2}$ is there because only half the molecules at any given moment are heading toward the wall in question, and $v$ is the speed of the molecules in the gas. That speed, in turn, can be inferred from the equipartition theorem, which says that each degree of freedom (in this case, motion in the x-direction) has an average energy of ${1 \over 2}RT$, and we know that kinetic energy is related to speed by $KE = {1 \over 2}Mv^2$ (where $M$ is the mass of a mole of molecules).
- $\delta KE$ is the change in kinetic energy of a mole of molecules hitting the receding wall (typical units: J/mol). As mentioned previously, for elastic collisions, each molecule's speed will be reduced by $2v_{wall}$ after it hits the wall.
- $A$ is the area of the wall getting hit (meter$^2$)
- $dt$ is the time interval (seconds) needed to for the wall to move out some small distance $dx$, i.e, $dt={dx \over v_{wall}}$. This is also related directly to the final volume, of course.

If these considerations don't convince you, you can try one more thing: do a unit analysis on the right hand side of Eq. 1. If the result has dimensions energy, we're on the right track!

For an expansion that's too big to be considered "infinitessimally small", we'll need to add up a series of tiny $dq_{kinetic}$ values appearing in Eq. 1, until the gas has expanded to our final, desired volume. Written in calculus-y terms, This accumulated heat is 

$$
q_{kinetic} = \int_{V_{init}}^{V_{final}} dq_{kinetic} \ \ \ \ (2)
$$

Simulated numerically, we'd write this as 

$$
q_{kinetic} \approx \sum_{V_{init}}^{V_{final}} dq_{kinetic} \ \ \ \ (3)
$$

which is what Numpy's np.sum function does. 

A couple of notes about implementing Eq. (3). First, we'd need to make sure that each $dq_{kinetic}$ in the sum is small enough that it's a good approximation to a bona fide (calculus-y) infinitessimal quantity. Second, we'd need to make sure that the starting volume for each tiny step is the ending volume of the previous step. (Were just mentioning this because we forgot to do that the first time through this.)

## A proper kinetic-molecular approach

The foregoing has a distinct numerical/algorithmic air to it. It will give results that are as accurate as you please, as long as you make the step size built into Eq. (3) small enough. But the development also shows us how to use kinetic-molecular theory to get an analytical result. Here are some key relationships:

$$
\delta KE = {\partial KE \over \partial v} \delta v \ \ \ \ (4)
$$

where ${\partial KE \over \partial v}$ can be got from taking the derivative of $KE = {1 \over 2}Mv^2$, and $\delta v=2v_{wall}$. If you put all this together, and notice that $Adx=dV$, you can come up with

$$
dq_{kinetic,proper} =  n M v^2 {dV \over V} \ \ \ \ (5)
$$

This expression, integrated from initial to final volumes, gives

$$
q_{kinetic,proper} = n M v^2 ln \bigl ({V_{final} \over V_{initial}} \bigr ) \ \ \ \ (6)
$$

where we can justify pulling $v^2$ out of the integral because we're insisting that this be an isothermal process.

## Regular old thermodynamic theory

And what does Thermodynamics have to say about this? Going back to a very small expansion, we can say that the infinessimal work done by an expanding gas is $dw = -P_{ex}dV$. Under isothermal and reversible conditions, for an ideal gas, and invoking the 1st Law, this translates to 
 
$$
dq_{thermo} = nRT {dV \over V} \ \ \ \ (7)
$$

For an expansion over many such steps, we can integrate over $V$, giving

$$
q_{thermo} = nRT \ ln \bigl ({V_{final} \over V_{initial}} \bigr ) \ \ \ \ (8)
$$

## Learning goals
The main learning goals of this exercise are 
1. I can provide a qualitative, molecular-level explanation for why heat must flow into a gas as it expands isothermally and reversibly.
1. I can use kinetic-molecular theory to predict the heat that flows into an ideal gas as it expands reversibly and isothermally.
1. I can explain the basis for that calculation, including the equipartition theory, elastic collision theory, and the frequency-of-collision theory. 
1. I can show that this formulation matches predictions using the the 1st Law of Thermodynamics, both for expansions that are tiny, and large (via integration).

In [1]:
import pint; from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Constants
R = AssignQuantity(8.314,'joule/mol/K')
NA = AssignQuantity(6.02e23,'1/mol')
T = AssignQuantity(293,'K')
n = AssignQuantity(1,'mol')
M = AssignQuantity(32,'g/mol')

### Kinetic theory associated with isothermal processes

In [3]:
# We'll need the average kinetic energy in the x-direction, using the equipartition theorem
KE = 0.5*R*T;
KE.ito('kjoule/mol')
print(KE)

# RMS speed associated with that average
v = (2/M*KE)**.5
v.ito('m/s')
print(v)

1.2180010000000001 kilojoule / mole
275.90770648896347 meter / second


### Specifying the initial state of our gas

In [4]:
# Specify the initial volume of our gas 
Vinit = AssignQuantity(24,'liter')
Vinit.ito('dm^3')
print('Initial volume = ', Vinit)

# Specify the starting length in the x-direction
xinit = AssignQuantity(5,'decimeter')
print('Initial distance in the x-direction = ', xinit)

# Deduce the area in the y & z directions consistent with the initial x and volume
A = Vinit/xinit
print('Sides of the wall = ', A**.5)

Initial volume =  24.000000000000007 decimeter ** 3
Initial distance in the x-direction =  5 decimeter
Sides of the wall =  2.1908902300206647 decimeter


### Hybrid kinetic-molecular theory of a tiny (infinitessimal) expansion of an ideal gas
Here, we construct all the factors that go into Eq. 1, and use those factors to find $dq_{kinetic}$.

In [5]:
# We'll need the starting mole density of our gas
rho = n/Vinit
rho.ito('mol/m^3')
print('Gas density = ', rho)

# Specify the distance the wall will move out (should be small, on the order of millimeters)
dx = AssignQuantity(10,'millimeter')
xfinal = xinit+dx
print('Final distance in the x-direction = ', xfinal)

# Calculate the increase in volume
dV = dx*A
print('Increase in volume = ', dV)

# Also specify the wall speed -- this should be small compared to the average molecular speed (for reversibility)
wallspeed = AssignQuantity(0.01,'m/s')

# Change in speed of a molecule hitting the wall
delta_v = 2*wallspeed
print('Change in speed = ', delta_v)

# Collision frequency if half the molecules (moving at the rms speed) are moving toward the receding wall
Z = rho/2*v
print('Collision frequency = ', Z)

# Resulting change in KE per mole of molecules hitting the wall
delta_KE = .5*M*((v-delta_v)**2-v**2)
delta_KE.ito('kilojoule/mol')
print('Change in KE per mole of collisions =', delta_KE)

# How long it takes to move that distance
dt = dx/wallspeed
dt.ito('sec')
print('Infinitessimal time needed to expand out a bit = ', dt)

# Heat required to compensate for the kinetic energy that is lost as molecules bounce off a receding wall
dq_kinetic = -delta_KE*Z*A*dt
dq_kinetic.ito('joule')
print(dq_kinetic)

Gas density =  41.66666666666665 mole / meter ** 3
Final distance in the x-direction =  5.1 decimeter
Increase in volume =  48.000000000000014 decimeter ** 2 * millimeter
Change in speed =  0.02 meter / second
Collision frequency =  5748.07721852007 mole / meter ** 2 / second
Change in KE per mole of collisions = -0.00017657453215285204 kilojoule / mole
Infinitessimal time needed to expand out a bit =  1.0 second
48.718274190655144 joule


### Thermodynamics of a tiny (infinitessimal) expansion
Below we calculate the infinitessimal amount of heat that has to go in to maintain isothermal conditions for reversible expansion of an ideal gas by a tiny amount ($dV$).

In [6]:
# According to thermo (infinitessimal)
dq_thermodynamic = n*R*T*dV/Vinit
dq_thermodynamic.ito('joule')
print(dq_thermodynamic)

48.72004 joule


### Pause for analysis
How close (%) does your infinitessimal heat amount obtained from kinetic theory come to your thermodynamic result?

In [7]:
error = (dq_kinetic-dq_thermodynamic)/dq_thermodynamic*100
print('Error = ', error)

Error =  -0.003624400441488047 dimensionless


### A streamlined calculation of $dq_{kinetic}$
To integrate our results, it'll be very handy to have a "functionalized" way of computing the infinitessimal heating for a small expansion. We do that in the cell below, making use of Eq. 5 in the Introduction.

In [8]:
def dq_calculator(n, M, v, V, dx, A):
    ###  This is a functionalized computation of the infinitessimal heating for a small expansion. ###
    
    dV = A*dx
    dq_kinetic = n*M*v**2*dV/V
    dq_kinetic.ito('joule')
    
    return dq_kinetic

# Testing the algorithm -- hopefully we get the same dq as what we got before
dq_kinetic_test = dq_calculator(n, M, v, Vinit, dx, A)
print('dq (test) = ', dq_kinetic_test)

dq (test) =  48.72004000000001 joule


### Numerical integration of our kinetic-molecular results
Assuming we're happy with our functionalized dq-calculator, we'll put it in a loop to simulate many steps -- see Eq. 3 in the Introduction.

In [13]:
# Specify how much the wall will move out (can be big because we're going to integrate!)
deltax = AssignQuantity(100,'centimeter')
print('Moving the wall from ', xinit, ' to ', xinit+deltax) 

# Also need to specify the number of steps, and the dx associated with each step
nsteps = 100
dx = deltax/nsteps

# Loop over our steps, updating our starting volume and recording the heat as we go along
dq_kinetic = AssignQuantity([],'joule')
for i in range(nsteps):
    Vinit_this_step = Vinit + i*dx*A
    dq_kinetic_this_step = dq_calculator(n, M, v, Vinit_this_step, dx, A)
    dq_kinetic = np.append(dq_kinetic, dq_kinetic_this_step)

# This is the numerical equivalent of integrating over all the steps 
q_kinetic = np.sum(dq_kinetic)
print('q_kinetic from numerical integration = ', q_kinetic)

Moving the wall from  5 decimeter  to  15.0 decimeter
2692.5339203831772 joule


### Analytical integration of our kinetic-molecular results 
See Eq. 6.

In [15]:
Vfinal = Vinit + deltax*A
q_kinetic_proper = n*M*v**2*np.log(Vfinal/Vinit)
q_kinetic_proper.ito('joule')
print('q_kinetic from analytical integration = ', q_kinetic_proper)

q_kinetic from analytical integration =  2676.2217324200933 joule


### Integrating our results -- thermodynamic theory
See Eq. 8.

In [10]:
# According to thermo (integrated)
Vfinal = Vinit + A*deltax
q_thermodynamic = n*R*T*np.log(Vfinal/Vinit)
print(q_thermodynamic)

2676.221732420093 joule


### Pause for analysis
How close (%) did your numerical result from kinetic theory come to your thermodynamic result? How close did your proper analytical result come? 

By the way, in situations like this, if an error comes in smaller than, say, $10^{01} \%$, we can assume that error is due to numerical issues (like how precise Python stores floating point numbers), so it's really zero.

In [18]:
# Error
error_numerical = (q_kinetic/q_thermodynamic-1)*100
print('Error of the numerical result (%) = ', error_numerical)

error_analytical = (q_kinetic_proper/q_thermodynamic-1)*100
print('Error of the analytical result (%) = ', error_analytical)

Error of the numerical result (%) =  0.609523036356685 dimensionless
Error of the analytical result (%) =  2.220446049250313e-14 dimensionless


### Refreshing and saving your code
1. Use the dropdown menu Kernel/Restart
2. Use the dropdown menu Cell/Run All Above
3. Under the "File" dropdown menu item in the upper left is a disk icon. Press it now to save your work (you can, do this at any time as you're working on an assignment, actually).

### Validating
This step will help ensure that you didn't miss something (although it's not a guarantee). Find the "Validate" button and press it. If there are any errors or warnings, fix them.

### Finishing up
Assuming all this has gone smoothly, carry out three more steps (but read this carefully before starting):
1. Close this notebook using the "File/Close and Halt" dropdown menu
1. Using the Assignments tab, submit this notebook
1. Press the Logout tab of the Home Page